##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 텐서플로 2.0 시작하기: 초보자용

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">구글 코랩(Colab)에서 실행하기</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">깃허브(GitHub) 소스 보기</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>   </td>
</table>

이 짧은 소개 글은 [Keras](https://www.tensorflow.org/guide/keras/overview)를 사용하여 다음을 수행합니다.

1. 사전에 빌드한 데이터세트를 로드합니다.
2. 이미지를 분류하는 신경망 머신 러닝 모델을 빌드합니다.
3. 이 신경망을 훈련합니다.
4. 모델의 정확도를 평가합니다.

이 문서는 [구글 코랩](https://colab.research.google.com/notebooks/welcome.ipynb)(Colaboratory) 노트북 파일입니다. 파이썬 프로그램을 브라우저에서 직접 실행할 수 있기 때문에 텐서플로를 배우고 사용하기 좋은 도구입니다:

1. 파이썬 런타임(runtime)에 연결하세요: 메뉴 막대의 오른쪽 상단에서 *CONNECT*를 선택하세요.
2. 노트북의 모든 코드 셀(cell)을 실행하세요: *Runtime* &gt; *Run all*을 선택하세요.

## TensorFlow 설정하기

시작하려면 TensorFlow를 프로그램으로 가져옵니다.

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

[MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)을 로드하여 준비합니다. 샘플 값을 정수에서 부동소수로 변환합니다:

참고: 자체 개발 환경을 사용하는 경우에 TensorFlow 2 패키지를 설치하려면 최신 `pip`로 업그레이드했는지 확인합니다. 자세한 내용은 [설치 가이드](https://www.tensorflow.org/install)를 참조합니다.

## 데이터세트 로드하기

[MNIST 데이터세트](http://yann.lecun.com/exdb/mnist/)를 로드하고 준비합니다. 샘플 데이터를 정수에서 부동 소수점 숫자로 변환합니다.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## 머신 러닝 모델 빌드하기

층을 차례대로 쌓아 `tf.keras.Sequential` 모델을 만듭니다. 훈련에 사용할 옵티마이저(optimizer)와 손실 함수를 선택합니다:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

각 예시에서 모델은 각 클래스에 대해 하나씩, [logits](https://developers.google.com/machine-learning/glossary#logits) 또는 [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) 스코어 벡터를 반환합니다.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

`tf.nn.softmax` 함수는 다음과 같이 이러한 로짓을 각 클래스에 대한 *확률*로 변환합니다. 

In [ ]:
tf.nn.softmax(predictions).numpy()

참고: `tf.nn.softmax` 함수를 네트워크의 마지막 레이어에 대한 활성화 함수로 베이킹할 수 있습니다. 이렇게 하면 모델 출력을 더 직접적으로 해석할 수 있지만 이 접근법은 소프트맥스 출력을 사용할 경우 모든 모델에 대해 정확하고 수치적으로 안정적인 손실 계산을 제공하는 것이 불가능하므로 권장하지 않습니다. 

`losses.SparseCategoricalCrossentropy`를 사용하여 로짓의 벡터와 `True` 인덱스를 사용하고 각 예시에 대해 스칼라 손실을 반환하는 훈련용 손실 함수를 정의합니다.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

이 손실은 실제 클래스의 음의 로그 확률과 같습니다. 모델이 올바른 클래스를 확신하는 경우 손실은 0입니다.

이 훈련되지 않은 모델은 무작위에 가까운 확률(각 클래스에 대해 1/10)을 제공하므로 초기 손실은 `-tf.math.log(1/10) ~= 2.3`에 근접해야 합니다.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

훈련을 시작하기 전에 Keras `Model.compile`을 사용하여 모델을 구성하고 컴파일합니다. [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) 클래스를 `adam`으로 설정하고 `loss`를 앞에서 정의한 `loss_fn` 함수로 설정합니다. `metrics` 매개변수를 `accuracy`로 설정하여 모델에 대해 평가할 메트릭을 지정합니다.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## 모델 훈련 및 평가하기

모델을 훈련하고 평가합니다: 

In [ ]:
model.fit(x_train, y_train, epochs=5)

`Model.evaluate` 메서드는 일반적으로 "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" 또는 "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)"에서 모델 성능을 확인합니다.

In [ ]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

훈련된 이미지 분류기는 이 데이터셋에서 약 98%의 정확도를 달성합니다. 더 자세한 내용은 [TensorFlow 튜토리얼](https://www.tensorflow.org/tutorials/)을 참고하세요.

모델이 확률을 반환하도록 하려면 다음과 같이 훈련된 모델을 래핑하고 여기에 소프트맥스를 첨부할 수 있습니다.

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

## 결론

축하합니다! [Keras](https://www.tensorflow.org/guide/keras/overview) API를 사용하는 사전에 빌드한 데이터세트를 사용하여 머신 러닝 모델을 훈련했습니다.

Keras를 사용하는 더 많은 예시는 [튜토리얼](https://www.tensorflow.org/tutorials/keras/)을 확인하세요. Keras로 모델을 빌드하는 방법에 대해 자세히 알아보려면 [가이드](https://www.tensorflow.org/guide/keras)를 읽어보세요. 데이터 로드 및 준비에 대해 자세히 알아보려면 [이미지 데이터 로드](https://www.tensorflow.org/tutorials/load_data/images) 또는 [CSV 데이터 로드](https://www.tensorflow.org/tutorials/load_data/csv)에 대한 튜토리얼을 참조하세요.
